In [1]:
import open3d as o3d
import numpy as np
import copy

print(o3d.__version__)
file_path = 'Data_collection/second_floor/'
image_number = 68
first_image = 0
voxel_size = 0.002
threshold = 0.002  / 1000000
max_iteration = 100

0.12.0


In [2]:
def preprocess_point_cloud(pcd, voxel_size):
#     print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
#     print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
#     print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [3]:
def prepare_dataset(image, voxel_size):
    # Create target point cloud
    file_name = '{:04d}'.format(image)
    pcd = o3d.io.read_point_cloud(file_path + "pcd/" + file_name + ".pcd")

#     print(":: Load target point cloud and disturb initial pose.")
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
#     source.transform(trans_init)

    # Extract fpfh
    pcd_down, pcd_fpfh = preprocess_point_cloud(pcd, voxel_size)
    return pcd, pcd_down, pcd_fpfh

In [4]:
# Global ICP
# RANSAC registration
def execute_global_registration(source, target, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
#     print(":: RANSAC registration on downsampled point clouds.")
#     print("   Since the downsampling voxel size is %.3f," % voxel_size)
#     print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source, target, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result

In [5]:
# Create target point cloud

target, target_down, target_fpfh = prepare_dataset(first_image, voxel_size)
aggr_pcd = target

In [6]:
# Transform all the pic
from registration import ICPSVD

transformations = []
trajectories = []
trajectories.append([0,0,0,1])

for index in range(first_image + 1, image_number + 1):
    print("Pic: {0:04d}".format(index))
    # Load point cloud and preprocessing
    source, source_down, source_fpfh = prepare_dataset(index, voxel_size)
    
    # Global registration
    result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
    trans_init = result_ransac.transformation

    # Local registration
    reg_SVD = ICPSVD(target_down, source_down, trans_init, threshold, max_iteration)

    # Update the transformations list and transform the current pcd to G_0
    transformations.append(reg_SVD)
    t = np.identity(4)
    for trans in transformations:
        t = np.dot(t, trans)
    # transofrm the source to G_0
    s = copy.deepcopy(source)
    s.transform(t)
    # Trajectory
    transition = np.append(reg_SVD[0:3,3], 1)
    trajectories.append(np.dot(t, transition))
    
    # Aggregate the result
    aggr_pcd += s
    
    target = source
    target_down = source_down
    target_fpfh = source_fpfh
    

Pic: 0001
Pic: 0002
Pic: 0003
Pic: 0004
Pic: 0005
Pic: 0006
Pic: 0007
Pic: 0008
Pic: 0009
Pic: 0010
Pic: 0011
Pic: 0012
Pic: 0013
Pic: 0014
Pic: 0015
Pic: 0016


/home/daniellin/anaconda3/envs/habitat/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/daniellin/anaconda3/envs/habitat/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/daniellin/anaconda3/envs/habitat/lib/python3.6/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/home/daniellin/Documents/habitat/hw1/files_of_ta/registration.py:59: RuntimeWarning: invalid value encountered in true_divide
  return indxSum/np.size(index,0)


Pic: 0017
Pic: 0018
Pic: 0019
Pic: 0020
Pic: 0021
Pic: 0022
Pic: 0023
Pic: 0024
Pic: 0025
Pic: 0026
Pic: 0027
Pic: 0028
Pic: 0029
Pic: 0030
Pic: 0031
Pic: 0032
Pic: 0033
Pic: 0034
Pic: 0035
Pic: 0036
Pic: 0037
Pic: 0038
Pic: 0039
Pic: 0040
Pic: 0041
Pic: 0042
Pic: 0043
Pic: 0044
Pic: 0045
Pic: 0046
Pic: 0047
Pic: 0048
Pic: 0049
Pic: 0050
Pic: 0051
Pic: 0052
Pic: 0053
Pic: 0054
Pic: 0055
Pic: 0056
Pic: 0057
Pic: 0058
Pic: 0059
Pic: 0060
Pic: 0061
Pic: 0062
Pic: 0063
Pic: 0064
Pic: 0065
Pic: 0066
Pic: 0067
Pic: 0068


In [7]:
# Flip it, otherwise the pointcloud will be upside down
aggr_pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
aggr_pcd_down = aggr_pcd.voxel_down_sample(voxel_size / 5)

In [8]:
# o3d.visualization.draw_geometries([aggr_pcd_down])

In [9]:
# Remove the roof
no_roof_pcd = copy.deepcopy(aggr_pcd_down)
pts = np.asarray(no_roof_pcd.points)
crs = np.asarray(no_roof_pcd.colors)
valid = pts[:,1] < 0

no_roof_pcd.points = o3d.utility.Vector3dVector(pts[valid])
no_roof_pcd.colors = o3d.utility.Vector3dVector(crs[valid])
# o3d.visualization.draw_geometries([no_roof_pcd])

In [10]:
points = []
lines = []
line_index = 1

with open(file_path + "GT_pose.txt", "r") as fp:
    line= fp.readline()
    x, y, z, rw, rx, ry, rz= line.split(" ")
    x0 = float(x) 
    y0 = float(y)
    z0 = float(z)
    points.append([(x0 - x0), (y0 - y0), (z0 - z0)])
    line= fp.readline()
    while line:
        x, y, z, rw, rx, ry, rz= line.split(" ")
        x = float(x) - x0
        y = float(y) - y0
        z = float(z) - z0
        points.append([x*0.0255, y*0.0255, z*0.0255])
        lines.append([line_index - 1, line_index])
        line_index += 1
        line = fp.readline()
print(len(points))
fp.close()

colors = [[1, 0, 0] for i in range(len(lines))]
gt_line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(points),
    lines=o3d.utility.Vector2iVector(lines),
)
gt_line_set.colors = o3d.utility.Vector3dVector(colors)
# o3d.visualization.draw_geometries([gt_line_set, no_roof_pcd])

69


In [11]:
points = []
lines = []
line_index = 0
# points.append(p)
for tra in trajectories:
    # multiply by transformation
#     p += -trans[0:3,3]
    tra = np.dot([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]], tra)
    points.append(tra[0:3])
    lines.append([line_index, line_index + 1])
    line_index += 1
del lines[-1]

colors = [[0, 0, 1] for i in range(len(lines))]
icp_line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(points),
    lines=o3d.utility.Vector2iVector(lines),
)
icp_line_set.colors = o3d.utility.Vector3dVector(colors)
# o3d.visualization.draw_geometries([icp_line_set, no_roof_pcd])

In [12]:
o3d.visualization.draw_geometries([gt_line_set, icp_line_set, no_roof_pcd])
